# Pseudobulk

Here I create pseudobulk data from some specific clusters expressin "Ntn4" gene.

The result would be an expression matrix of samples and genes for each cluster.


In [1]:
import scanpy as sc
import anndata as ad

import pandas as pd
import numpy as np

In [3]:
sc.logging.print_version_and_date()
sc.logging.print_header()

Running Scanpy 1.10.3, on 2024-12-05 12:13.
scanpy==1.10.3 anndata==0.10.9 umap==0.5.7 numpy==2.0.2 scipy==1.14.1 pandas==2.2.3 scikit-learn==1.5.2 statsmodels==0.14.4 igraph==0.11.8 pynndescent==0.5.13


## Prepare data

### read data

In [4]:
adata = sc.read_h5ad("results/clustered_annotated_final.h5ad.gz")

In [5]:
adata

AnnData object with n_obs × n_vars = 36553 × 21893
    obs: 'data', 'status', 'replicate', 'disease', 'condition', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts', 'doublet_score', 'predicted_doublet', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable_nbatches', 'highly_variable_intersection', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'condition_colors', 'data_colors', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [6]:
adata_healthy = adata.copy()
adata_healthy = adata_healthy[adata_healthy.obs["disease"] == "healthy", :]
adata_healthy

View of AnnData object with n_obs × n_vars = 20233 × 21893
    obs: 'data', 'status', 'replicate', 'disease', 'condition', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts', 'doublet_score', 'predicted_doublet', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable_nbatches', 'highly_variable_intersection', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'condition_colors', 'data_colors', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [8]:
# List of desired cell types
cell_types_to_extract = ["Capi aerocyte", "Vein EC", "Capi", "Advential fibro", "Pericyte", "Myofibroblast", "Alveolar epi type 2", "Lymphatic vessel EC"]

# Check that your cell type annotation is stored in adata.obs
cell_type_column = "cell_type"

# Subset the AnnData object
adata_subset = adata_healthy[adata_healthy.obs[cell_type_column].isin(cell_types_to_extract)].copy()

# Verify the result
print(f"Original adata shape: {adata_healthy.shape}")
print(f"Subset adata shape: {adata_subset.shape}")


Original adata shape: (20233, 21893)
Subset adata shape: (4969, 21893)


# pseudobulk

In [9]:
import pandas as pd
import numpy as np

# Define columns in `adata_subset.obs` for sample and cell type
sample_column = "replicate"  # Replace with the column name for sample info
cell_type_column = "cell_type"  # Replace with the column name for cell type info

# Ensure raw counts are available in adata.layers['counts']
if "counts" not in adata_subset.layers:
    raise ValueError("Raw counts not found in `adata_subset.layers['counts']`.")

# Create an empty dictionary to store pseudobulk matrices
pseudobulk_data = {}

# Iterate through each cell type
for cell_type in adata_subset.obs[cell_type_column].unique():
    # Subset the data for the current cell type
    cell_type_data = adata_subset[adata_subset.obs[cell_type_column] == cell_type]

    # Extract raw counts from the counts layer
    counts_matrix = cell_type_data.layers["counts"]

    # If the data is sparse, convert it to dense
    if not isinstance(counts_matrix, np.ndarray):
        counts_matrix = counts_matrix.toarray()

    # Create a DataFrame with counts and sample info
    counts_df = pd.DataFrame(
        counts_matrix,
        index=cell_type_data.obs[sample_column],  # Rows correspond to samples
        columns=adata_subset.var_names                  # Columns correspond to genes
    )

    # Aggregate counts by summing across samples for each gene
    pseudobulk_matrix = counts_df.groupby(counts_df.index).sum()

    # Store the matrix in the dictionary
    pseudobulk_data[cell_type] = pseudobulk_matrix

# Save pseudobulk matrices to CSV files
for cell_type, matrix in pseudobulk_data.items():
    matrix.to_csv(f"results/{cell_type}_pseudobulk.csv")
    print(f"Pseudobulk matrix saved for {cell_type}.")


/tmp/ipykernel_909199/3770544564.py:35: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pseudobulk_matrix = counts_df.groupby(counts_df.index).sum()


Pseudobulk matrix saved for Capi.
Pseudobulk matrix saved for Capi aerocyte.
Pseudobulk matrix saved for Pericyte.
Pseudobulk matrix saved for Vein EC.
Pseudobulk matrix saved for Myofibroblast.
Pseudobulk matrix saved for Advential fibro.
Pseudobulk matrix saved for Alveolar epi type 2.
Pseudobulk matrix saved for Lymphatic vessel EC.


In [15]:
print(cell_type)
matrix

Lymphatic vessel EC


,Gm19938,Rp1,Sox17,Gm37587,Gm37323,Mrpl15,Lypla1,Tcea1,Rgs20,Atp6v1h,...,Csprs,AC132444.6,Vamp7,Spry3,Tmlhe,CR974586.4,4933409K07Rik,Gm10931,CAAA01147332.1,AC149090.1
replicate,,,,,,,,,,,,,,,,,,,,,
KO-Healthy-rep1,0.0,0.0,8.0,0.0,0.0,11.0,10.0,29.0,0.0,5.0,...,0.0,0.0,6.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
KO-Healthy-rep2,1.0,0.0,5.0,0.0,0.0,14.0,20.0,27.0,0.0,12.0,...,0.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0
WT-Healthy-rep1,3.0,0.0,3.0,1.0,0.0,9.0,16.0,26.0,1.0,13.0,...,0.0,0.0,7.0,0.0,2.0,0.0,0.0,0.0,1.0,4.0
WT-Healthy-rep2,3.0,0.0,7.0,0.0,0.0,16.0,12.0,52.0,0.0,10.0,...,0.0,0.0,13.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0
